In [1]:
from itertools import combinations

In [2]:
def intro():
    print('+---------------------------------+')
    print('|  STRING FROM GRAMMAR GENERATOR  |')
    print('|          G = (N,Ʃ,P,S)          |')
    print('|                                 |')
    print('| N - nonterminal symbols         |')
    print('| Ʃ - terminal symbols            |')
    print('| P - production rules            |')
    print('| S - start symbol                |')
    print('+---------------------------------+')

In [3]:
def read():
    
    global terminal_symbols
    global nonterminal_symbols
    global rules
    
    file = open('grammar.txt', 'r')

    line = file.readline()
    terminal_symbols = []
    while line[:-1] != 'next':
        terminal_symbols.append(line[:-1])
        line = file.readline()

    line = file.readline()
    nonterminal_symbols = []
    while line[:-1] != 'next':
        nonterminal_symbols.append(line[:-1])
        line = file.readline()

    line = file.readline()
    rules = []
    while line[:-1] != 'next':
        rules.append(line[:-1])
        line = file.readline()

    file.close()

In [4]:
def output_intro_data():

    global terminal_symbols
    global nonterminal_symbols
    global rules 

    print('The grammer is:')
    print('G(N,Ʃ,P,S)')

    output = 'Ʃ = {'
    for x in terminal_symbols:
        output += x + ', '
    output = output[:(len(output) - 2)]
    output += '}'
    print(output)

    output = 'N = {'
    for x in nonterminal_symbols:
        output += x + ', '
    output = output[:(len(output) - 2)]
    output += '}'
    print(output)

    output = 'P = {\n'
    for x in rules:
        output += '\t' + x + '\n'
    output += '}'
    print(output)

In [5]:
def grammar_to_dictionary():
    
    global rules_dic
    
    rules_dic = {}
    for rule in rules:
        if not (rule[0] in rules_dic):
            rules_dic[rule[0]] = []
        start = rule.index('>') + 2
        right_production_rule = rule[start:len(rule)]
        symbols = []
        for symbol in right_production_rule:
            symbols.append(symbol)
        rules_dic[rule[0]].append(symbols)
    print(rules_dic)

In [6]:
def eliminations_of_e_productions():
    
    global terminal_symbols
    global rules_dic
    global new_rules_dic
    global nonterminal_symbols
    global renamings
    
    used_empty_symbols = []
    empty_symbols = []
    for key in rules_dic:
        for rule in rules_dic[key]:
            if len(rule) == 1 and rule[0] == 'e':
                empty_symbols.append(key)
                rules_dic[key].remove(rule)

    while len(empty_symbols) != 0: 
        new_rules_dic = {}
        for key in rules_dic:
            new_rules_dic[key] = []

        for empty_symbol in empty_symbols:
            used_empty_symbols.append(empty_symbol)
            for key in rules_dic:
                for rule in rules_dic[key]:
                    new_rules_dic[key].append(rule)
                    if len(rule) == 1 and rule[0] in nonterminal_symbols:
                        renaming = []
                        renaming.append(key)
                        renaming.append(rule[0])
                        if renaming not in renamings:
                            renamings.append(renaming)
                    if empty_symbol in rule:
                        new_rule = []
                        for x in rule:
                            new_rule.append(x)
                        empty_symbol_count = new_rule.count(empty_symbol)
                        elements_of_combinations = []
                        all_combinations = []
                        for x in range(empty_symbol_count):
                            elements_of_combinations.append(x)
                        for x in range(1, empty_symbol_count + 1):
                            all_combinations += combinations(elements_of_combinations, x)
                        for combination in all_combinations:
                            new_rule = []
                            for x in rule:
                                new_rule.append(x)
                            for index_of_combination in range(len(combination)):
                                count = -1
                                for element_index in range(len(new_rule)):
                                    if new_rule[element_index] == empty_symbol:
                                        count += 1
                                    if count == (combination[index_of_combination] - index_of_combination):
                                        break
                                new_rule.pop(element_index)
                            new_rules_dic[key].append(new_rule)
                            if len(new_rule) == 1 and new_rule[0] in nonterminal_symbols:
                                renaming = []
                                renaming.append(key)
                                renaming.append(new_rule[0])
                                if renaming not in renamings:
                                    renamings.append(renaming)


        rules_dic = {}
        for key in new_rules_dic:
            rules_dic[key] = new_rules_dic[key]
        new_rules_dic = {}

        empty_symbols = []
        for key in rules_dic:
            for rule in rules_dic[key]:
                if len(rule) == 0 and not(key in used_empty_symbols):
                    empty_symbols.append(key)
                    rules_dic[key].remove(rule)
                elif len(rule) == 0 and key in used_empty_symbols:
                    rules_dic[key].remove(rule)

    #eliminate duplicates
    for key in rules_dic:
        key_rules = []
        for rule in rules_dic[key]:
            if rule not in key_rules:
                key_rules.append(rule)
        rules_dic[key] = key_rules
    print(rules_dic) 

In [7]:
def eliminations_of_renamings():
    
    global rules_dic
    global renamings
    global nonterminal_symbols
    global terminal_symbols
    
    for key in rules_dic:
        for rule in rules_dic[key]:
            if len(rule) == 1 and rule[0] in nonterminal_symbols:
                rules_dic[key].remove(rule)
    print(renamings)

    #lucreaza corect
    is_change = True
    while is_change == True:
        is_change = False
        for renaming in renamings:
            first = renaming[0]
            second = renaming[1]
            for rule in rules_dic[second]:
                if rule not in rules_dic[first]:
                    is_change = True
                    rules_dic[first].append(rule)
    print(rules_dic)

In [8]:
def eliminations_of_nonproductive_symbols():
    
    global rules_dic
    global renamings
    global nonterminal_symbols
    global terminal_symbols
    global new_rules_dic
    global productive
    
    productive = []
    for key in rules_dic:
        for rule in rules_dic[key]:
            if len(rule) == 1 and rule[0] in terminal_symbols and key not in productive:
                productive.append(key)
    is_update = True
    while is_update:
        is_update = False
        for key in rules_dic:
            if key not in productive:
                for rule in rules_dic[key]:
                    transfer_to_productive = True
                    for symbol in rule:
                        if symbol not in terminal_symbols and symbol not in productive:
                            transfer_to_productive = False
                    if transfer_to_productive == True:
                        productive.append(key)
                        is_update = True
    nonproductive = []
    for symbol in nonterminal_symbols:
        if symbol not in productive:
            nonproductive.append(symbol)

    print(productive)
    print(nonproductive)

    for symbol in nonproductive:
        del rules_dic[symbol]
        for key in rules_dic:
            for rule in rules_dic[key]:
                if symbol in rule:
                    rules_dic[key].remove(rule)

    print(rules_dic)

In [9]:
def eliminations_of_inaccesibile_symbols():
    
    global rules_dic
    global renamings
    global nonterminal_symbols
    global terminal_symbols
    global new_rules_dic
    global productive

    acces = []
    acces.append('S')
    for symbol in acces:
        for rule in rules_dic[symbol]:
            for char in rule:
                if char in nonterminal_symbols and char not in acces:
                    acces.append(char)
    print(acces)
    inacces = []
    for symbol in productive:
        if symbol not in acces:
            inacces.append(symbol)
    for symbol in inacces:
        del rules_dic[symbol]

    print(rules_dic)

In [10]:
def terminal_symbols_transformations():
    
    global rules_dic
    global renamings
    global nonterminal_symbols
    global terminal_symbols
    global new_rules_dic

    terminal_to_nonterminal = []
    for symbol in terminal_symbols:
        terminal_to_nonterminal.append('X' + str(ord(symbol) - 96))
        rules_dic[terminal_to_nonterminal[-1]] = []
        rule = []
        rule.append(symbol)
        rules_dic[terminal_to_nonterminal[-1]].append(rule)
    print(terminal_to_nonterminal)

    for key in rules_dic:
        for rule in rules_dic[key]:
            if len(rule) >= 2:
                for index in range(len(rule)):
                    if rule[index] in terminal_symbols:
                        rule[index] = 'X' + str(ord(rule[index]) - 96)
    print(rules_dic)

In [11]:
def nonterminal_symbols_transformations():
    
    global rules_dic
    global renamings
    global nonterminal_symbols
    global terminal_symbols
    global new_rules_dic

    index = 1
    nonterminal_to_nonterminal = []
    new_rules_dic = {}
    for key in rules_dic:
        new_rules_dic[key] = []
        for rule in rules_dic[key]:
            if len(rule) > 2:
                to_change = []
                for symbol in rule:
                    to_change.append(symbol)
                to_change.pop(-1)
                nonterminal_to_nonterminal.append('Y' + str(index))
                index += 1
                new_rule = []
                new_rule.append(nonterminal_to_nonterminal[index - 2])
                new_rule.append(rule[-1])
                #print(new_rule)
                new_rules_dic[key].append(new_rule)
                new_rules_dic[nonterminal_to_nonterminal[index - 2]] = []
                aux = []
                for symbol in to_change:
                    aux.append(symbol)
                new_rules_dic[nonterminal_to_nonterminal[index - 2]].append(aux)
                #print(nonterminal_to_nonterminal[index - 2], to_change )
                while len(to_change) > 2:
                    to_change.pop(-1)
                    nonterminal_to_nonterminal.append('Y' + str(index))
                    index += 1
                    new_rule = []
                    new_rule.append(nonterminal_to_nonterminal[index - 2])
                    #print(nonterminal_to_nonterminal[index - 3], new_rules_dic[nonterminal_to_nonterminal[index - 3]])
                    new_rule.append(new_rules_dic[nonterminal_to_nonterminal[index - 3]][0][-1])
                    print(new_rule)
                    new_rules_dic[nonterminal_to_nonterminal[index - 3]][0] = new_rule
                    new_rules_dic[nonterminal_to_nonterminal[index - 2]] = [] 
                    aux = []
                    for symbol in to_change:
                        aux.append(symbol)
                    new_rules_dic[nonterminal_to_nonterminal[index - 2]].append(aux) 
            else:
                new_rules_dic[key].append(rule)

    index = len(nonterminal_to_nonterminal) - 1
    while index >= 0:
        current = index - 1
        symbol1 = nonterminal_to_nonterminal[index]
        while current >= 0:
            symbol2 = nonterminal_to_nonterminal[current]
            if symbol1 in new_rules_dic and new_rules_dic[symbol1] == new_rules_dic[symbol2]:
                for key in new_rules_dic:
                    for rule in new_rules_dic[key]:
                        for num in range(len(rule)):
                            if rule[num] == symbol1:
                                rule[num] = symbol2
                del new_rules_dic[symbol1]
            current -= 1
        index -= 1

    print(new_rules_dic)

In [12]:
def display_CNF():
    
    global new_rules_dic
    
    for key in new_rules_dic:
        for rule in new_rules_dic[key]:
            rule_string = ''
            for symbol in rule:
                rule_string += symbol
            print(str(key), ' -> ', rule_string)

In [13]:
terminal_symbols = []
nonterminal_symbols = []
rules = []
rules_dic = {}
renamings = []
new_rules_dic = {}
productive = []

intro()
read()
output_intro_data()
print('----------------------------------------------------------------------------')
grammar_to_dictionary()
print('----------------------------------------------------------------------------')
eliminations_of_e_productions()
print('----------------------------------------------------------------------------')
eliminations_of_renamings()
print('----------------------------------------------------------------------------')
eliminations_of_nonproductive_symbols()
print('----------------------------------------------------------------------------')
eliminations_of_inaccesibile_symbols()
print('----------------------------------------------------------------------------')
terminal_symbols_transformations()
print('----------------------------------------------------------------------------')
nonterminal_symbols_transformations()
print('----------------------------------------------------------------------------')
display_CNF()

+---------------------------------+
|  STRING FROM GRAMMAR GENERATOR  |
|          G = (N,Ʃ,P,S)          |
|                                 |
| N - nonterminal symbols         |
| Ʃ - terminal symbols            |
| P - production rules            |
| S - start symbol                |
+---------------------------------+
The grammer is:
G(N,Ʃ,P,S)
Ʃ = {a, b}
N = {S, A, B, C, D}
P = {
	S -> aB
	S -> bA
	S -> A
	A -> B
	A -> Sa
	A -> bBA
	A -> b
	B -> b
	B -> bS
	B -> aD
	B -> e
	D -> AA
	C -> Ba
}
----------------------------------------------------------------------------
{'S': [['a', 'B'], ['b', 'A'], ['A']], 'A': [['B'], ['S', 'a'], ['b', 'B', 'A'], ['b']], 'B': [['b'], ['b', 'S'], ['a', 'D'], ['e']], 'D': [['A', 'A']], 'C': [['B', 'a']]}
----------------------------------------------------------------------------
{'S': [['a', 'B'], ['a'], ['b', 'A'], ['b'], ['A']], 'A': [['B'], ['S', 'a'], ['a'], ['b', 'B', 'A'], ['b', 'B'], ['b', 'A'], ['b']], 'B': [['b'], ['b', 'S'], ['a', 'D'], 